# Lempel-Ziv-Welch (LZW) Compression Algorithm Demo

Encoding, decoding, and printing the LZW table

Author: Zhiyu An

Date: 13-Mar-2023

In [38]:
ALPHABET = {'A': 1, 
            'B': 2, 
            'C': 3}
# ALPHABET = {}
# ENGLISH = 'abcdefghijklmnopqrstuvwxyz ABCDEFGHIJKLMNOPQRSTUVWXYZ'
# for i in range(27):
#     ALPHABET[ENGLISH[i]] = i + 1
INPUT = 'ABABBABCABABBA'
# INPUT = 'the quick brown fox jumps over the lazy dog'

In [39]:
def encode(uncompressed):
    dictionary = ALPHABET.copy() # Prelude
    S, Output, S_history = "", [], [] # Initialize S and Output to none, and S_history for printing the table
    for C in uncompressed: # For each character C in uncompressed
        SC = S + C # Combine S and C
        if SC in dictionary: # Check if SC is in the dictionary
            S = SC # If so, set S to SC
            S_history.append(S) # Add S to the history
        else: # Otherwise
            Output.append(dictionary[S]) # Add the code of S to the output
            dictionary[SC] = len(dictionary) + 1 # Add SC to the dictionary and assign it a new code
            S = C # Replace S with C
            S_history.append(S) # Add S to the history
    if S: # Tail case
        Output.append(dictionary[S]) # Add the code of S to the output
    '''Print LZW table'''
    S_column = ['' for _ in range(len(ALPHABET))] + S_history # Get the S column
    C_column = ['' for _ in range(len(ALPHABET))] + list(uncompressed)[1:] + ['EOF'] # Get the C column
    Output_column = ['' for _ in range(len(ALPHABET))] + Output + ['' for _ in range(len(S_column))]# Get the Output column
    Code_column = [dictionary[i] for i in list(dictionary.keys())] + ['' for _ in range(len(S_column))] # Get the Code column
    String_column = list(dictionary.keys()) + ['' for _ in range(len(S_column))] # Get the String column
    print('S\tC\tOutput\tCode\tString') # Print the header
    for i in range(len(S_column)): # For each row
        print(f'{S_column[i]}\t{C_column[i]}\t{Output_column[i]}\t{Code_column[i]}\t{String_column[i]}') # Print the row
    return Output # Return the output


def decode(compressed):
    dictionary = ALPHABET.copy() # Prelude
    dictionary = dict(zip(dictionary.values(), dictionary.keys())) # reverse dictionary
    S = dictionary[compressed[0]] # Get the first character
    compressed = compressed[1:] # Remove the first character from the compressed string
    result = [S] # Set the result to the first character
    for code in compressed: # For each code in the compressed string
        if code in dictionary.keys(): # Check if the code is in the dictionary
            entry = dictionary[code] # If so, set entry to the code
        elif code == len(dictionary) + 1: # Otherwise, check if the code is the next code
            entry = S + S[0] # If so, set entry to S + S[0]
        else: # Otherwise
            raise ValueError('Corrupted') # Raise an error
        result.append(entry) # Add entry to the result
        dictionary[len(dictionary) + 1] = S + entry[0] # Add S + entry[0] to the dictionary
        S = entry # Set S to entry
    return result # Return the decoded string


compressed = encode(INPUT)
print('Output: ', ' '.join([str(i) for i in compressed]))
print('Input  : ', INPUT)
print('Decoded: ', ''.join([str(i) for i in decode(compressed=compressed)]))
print('Validated: ', INPUT == ''.join([str(i) for i in decode(compressed=compressed)]))

S	C	Output	Code	String
			1	A
			2	B
			3	C
A	B	1	4	AB
B	A	2	5	BA
A	B	4	6	ABB
AB	B	5	7	BAB
B	A	2	8	BC
BA	B	3	9	CA
B	C	4	10	ABA
C	A	6	11	ABBA
A	B	1		
AB	A			
A	B			
AB	B			
ABB	A			
A	EOF			
Output:  1 2 4 5 2 3 4 6 1
Input  :  ABABBABCABABBA
Decoded:  ABABBABCABABBA
Validated:  True
